# Data Enrichment - Tiling and Rotating 

In [1]:
 from google.colab import drive
 drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import re
import numpy as np
import subprocess
import cv2
import math
import shutil

In [ ]:
myPathLabel = '/content/drive/MyDrive/dataset/labels/validation'
myPathImages = '/content/drive/MyDrive/dataset/images/validation'

## Tiling

Wie im Abschnitt Model Evaluation erläutert, ist eine Möglichkeit zum Umgang mit großen Bilder das "Kacheln" (englisch tiling). Die besondere Herausforderung ist hierbei, dass sowohl die Bild-, wie auch die Labeldatein zerlegt werden müssen. Hierbei wird auf ein bereits vorhandenes Github-Repository von {cite:t}`YOLO_Dataset_Tiling` zurückgegriffen. 

In [ ]:
#! git clone https://github.com/slanj/yolo-tiling.git

In [ ]:
label_list = os.listdir(myPathLabel)
#image_list = os.listdir(myPathImages)

In [ ]:
image_list = list()
for x in label_list:
  png = str.split(x, ".")[0] + ".png"
  image_list.append(png)

In [ ]:
for file1 in image_list:
    src_path = '/content/drive/MyDrive/dataset/images/validation/' + file1
    dst_path = '/content/drive/MyDrive/dataset_tile/org/validation/' + file1
    shutil.copy(src_path, dst_path)
    #print('Copied')
for file1 in label_list:
    src_path = '/content/drive/MyDrive/dataset/labels/validation/' + file1
    dst_path = '/content/drive/MyDrive/dataset_tile/org/validation/' + file1
    shutil.copy(src_path, dst_path)
    #print('Copied')

Mit diesem Repository können nur Bilder mit Label (keine Backgroundbilder), deswegen werden diese herausgefiltert. 

In [ ]:
%cd yolo-tiling/

/content/yolo-tiling


Mit Aufruf der `tile_yolo.py` werden die Bilder und Labels im entsprechenden Ordner in Kacheln der Größe 640x640 Pixel zerlegt und in einem entsprechenden Zielordner ablegt. Dies wird für Trainings- und Validierungsdaten durchgeführt. 

In [ ]:
! python3 tile_yolo.py -source /content/drive/MyDrive/dataset_tile/org/validation -target /content/drive/MyDrive/dataset_tile/tiled/validation -ext .png -size 640

Streaming output truncated to the last 5000 lines.
[2 rows x 5 columns]
/content/drive/MyDrive/dataset_tile/tiled/validation/P1156_1_3.png
   class  ...         h
0      0  ...  0.292186
1      0  ...  0.256251
2      0  ...  0.272658
3      0  ...  0.321873

[4 rows x 5 columns]
/content/drive/MyDrive/dataset_tile/tiled/validation/P1156_1_4.png
   class  ...         h
0      0  ...  0.091847
1      0  ...  0.292186

[2 rows x 5 columns]
/content/drive/MyDrive/dataset_tile/tiled/validation/P1156_2_1.png
   class  ...         h
0      0  ...  0.252512

[1 rows x 5 columns]
/content/drive/MyDrive/dataset_tile/tiled/validation/P1156_2_2.png
   class  ...         h
0      0  ...  0.286719
1      0  ...  0.302345
2      0  ...  0.252512

[3 rows x 5 columns]
/content/drive/MyDrive/dataset_tile/tiled/validation/P1156_3_0.png
   class  ...         h
0      0  ...  0.451562
1      0  ...  0.167291

[2 rows x 5 columns]
/content/drive/MyDrive/dataset_tile/tiled/validation/P1156_3_1.png
   class

In [ ]:
tiled_list = os.listdir('/content/drive/MyDrive/dataset_tile/tiled/validation')

Die enstandene Ordnerstruktur (validation -> images) entspricht nicht der Zielstruktur (images -> validation). Aus diesem Grund werden die Dateien erneut verschoben. 

In [ ]:
tiled_img = list()
tiled_txt = list()
for file in tiled_list: 
  if str.split(file, ".")[1] == "png":
    tiled_img.append(file)
  elif str.split(file, ".")[1] == "txt":
    tiled_txt.append(file)

In [ ]:
for file1 in tiled_img:
    src_path = '/content/drive/MyDrive/dataset_tile/tiled/validation/' + file1
    dst_path = '/content/drive/MyDrive/dataset_tile/tiled/validation/images/' + file1
    shutil.move(src_path, dst_path)
    #print('Copied')
for file1 in tiled_txt:
    src_path = '/content/drive/MyDrive/dataset_tile/tiled/validation/' + file1
    dst_path = '/content/drive/MyDrive/dataset_tile/tiled/validation/labels/' + file1
    shutil.move(src_path, dst_path)
    #print('Copied')

## Tiling Background

Wie oben erwähnt, funktioniert das erwähnte Repo nur für Bilder mit Labels. Da die Backgroundbilder keine Labels haben, müssen diese seperat "getilt" werden. Dafür werden die Bibliotheken `cv2` und `math` genutzt.  

In [ ]:
def get_tiles(src_path,dst_path,file):
  img = cv2.cvtColor(cv2.imread(f"{src_path}{file}"),cv2.COLOR_BGR2RGB)
  img_shape = img.shape
  tile_size = (640, 640)
  offset = (640, 640)
  
  for i in list(range(int(math.ceil(img_shape[0]/(offset[1] * 1.0))))):
      for j in list(range(int(math.ceil(img_shape[1]/(offset[0] * 1.0))))):
          cropped_img = img[offset[1]*i:min(offset[1]*i+tile_size[1], img_shape[0]), offset[0]*j:min(offset[0]*j+tile_size[0], img_shape[1])]
          # Debugging the tiles
          cv2.imwrite(dst_path + "debug_" + str(i) + "_" + str(j) + file, cv2.cvtColor(cropped_img,cv2.COLOR_BGR2RGB))
  return (i+1)*(j+1)

Mit der oben aufgeführten Funktion werden die Bilder zerlegt. Da sich bei Bildern implizit um mehrdimensionale Listen handelt, werden diese entsprechend der gewünschten Teilung durch itteriert. 
Wichtig hier ist es, die Farbkanäle mit cv2.COLOR_BGR2RGB anzupassen. Da diese  durch die `cv2`-Bibliothek vertauscht werden. 

In [ ]:
dir_path = "/content/drive/MyDrive/dataset/labels/validation" #GoogleDrive Data Folders
img_path = "/content/drive/MyDrive/dataset/images/removed_background_validation/"
img_list_all = os.listdir(img_path)
labels_list_all  = os.listdir(dir_path)

In [ ]:
image_all = list()
for img in img_list_all:
  image_all.append(str.split(img, ".")[0])

In [ ]:
label_all = list()
for label in labels_list_all:
  label_all.append(str.split(label, ".")[0])

In [ ]:
background_img = list()
no_background_img = list()
for filename in image_all:
  pic_name = filename+".png" 
  if filename in label_all:
    no_background_img.append(pic_name)
  else: 
    background_img.append(pic_name)

Um die Backgroundbilder zu ermitteln, werden alle Bilder die keine Labeldatei haben in eine Liste gespeichert. 

In [ ]:
count_tiles = 0
source_path = img_path
dest_path = '/content/drive/MyDrive/dataset_tile/tiled/images/validation/'
for i in background_img:
  count_tiles += get_tiles(source_path,dest_path,i)
  if count_tiles >= 207:
    break

Anschließend werden die Backgroundbilder zerlegt bis 207 Kacheln entstanden sind. Die 207 entspricht der 10%-Anreicherung der Daten mit Backgroundbildern. 

## Rotate

Um die Anzahl an Instanzen der Klasse Helikopter zu erhöhen, werden die vorhandenen Bilder und Label zusätzlich zu den YOLOv5 interen Funktionen um einen beliebigen Winkel gedreht. 

In [ ]:
img_list = list()
text_list = list()
label_list = os.listdir(myPath)
for filename in label_list:
  path =  myPath + "/" + filename
  with open(path) as f:
    for lines in f:
      if (re.match("^1",lines)):
        png = str.split(filename, ".")[0] + ".png"
        txt = str.split(filename, ".")[0] + ".txt"
        text_list.append(txt)
        img_list.append(png)
        continue
img_list = np.unique(img_list)
label_list = np.unique(text_list)
# return img_list, text_list

Dafür werden im ersten Schritt die Datein mit Helikoptern über die Einträge in der Labeldatei ermittelt. 

Ähnlich wie bei dem Tiling ist auch beim Rotieren die Herausforderung das Label und Bilder rotiert werden müssen. Aus diesem Grund wird ebenfalls auf ein vorhandenes Gihtub-Repository zurückgegriffen. {cite:p}`Bbox_rotate`

In [ ]:
! git clone https://github.com/usmanr149/Yolo_bbox_manipulation.git

Cloning into 'Yolo_bbox_manipulation'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 7), reused 16 (delta 3), pack-reused 0
Unpacking objects: 100% (23/23), done.


Vorrausetzung für die Ausführung des Codes ist, dass sich die Bilder und Label im gleichen Ordner befinden. Aus diesem Grund müssen diese erneut verschoben werden. 

In [ ]:
for file1 in img_list:
    src_path = '/content/drive/MyDrive/dataset/images/train/' + file1
    dst_path = '/content/Yolo_bbox_manipulation/' + file1
    shutil.copy(src_path, dst_path)
    print('Copied')
for file1 in label_list:
    src_path = '/content/drive/MyDrive/dataset/labels/train/' + file1
    dst_path = '/content/Yolo_bbox_manipulation/' + file1
    shutil.copy(src_path, dst_path)
    print('Copied')

Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied
Copied


In [ ]:
%cd Yolo_bbox_manipulation

/content/Yolo_bbox_manipulation


Zum Rotieren wird  `rotate.py` aus dem erwähnten Repository über die `subprocess` -Bibliothek aufgerufen. Die Bibliothek ermöglicht den Terminalaufruf in einer For-Schleife. Für einen signifikaten Anstieg der Instanzanzahl der Helikopter wird jedes Bild sechsmal rotiert. Bei jeder Rotation wird ein zufälliger Winkel zwischen 1 und 359 Grad genutzt. 

In [ ]:
counter = 0
import subprocess
img_list = img_list[1:]
# myangle="65"
for x in img_list:
  for i in range(0,6):
    myangle = str(np.random.randint(1,359))
    cmd = "python rotate.py -i "+x+" -a "+myangle
    subprocess.call(cmd, shell=True)  # returns the exit code in unix
    counter += 1
    print('counter:', counter)

counter: 1
counter: 2
counter: 3
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 9
counter: 10
counter: 11
counter: 12
counter: 13
counter: 14
counter: 15
counter: 16
counter: 17
counter: 18
counter: 19
counter: 20
counter: 21
counter: 22
counter: 23
counter: 24
counter: 25
counter: 26
counter: 27
counter: 28
counter: 29
counter: 30
counter: 31
counter: 32
counter: 33
counter: 34
counter: 35
counter: 36
counter: 37
counter: 38
counter: 39
counter: 40
counter: 41
counter: 42
counter: 43
counter: 44
counter: 45
counter: 46
counter: 47
counter: 48
counter: 49
counter: 50
counter: 51
counter: 52
counter: 53
counter: 54
counter: 55
counter: 56
counter: 57
counter: 58
counter: 59
counter: 60
counter: 61
counter: 62
counter: 63
counter: 64
counter: 65
counter: 66
counter: 67
counter: 68
counter: 69
counter: 70
counter: 71
counter: 72
counter: 73
counter: 74
counter: 75
counter: 76
counter: 77
counter: 78
counter: 79
counter: 80
counter: 81
counter: 82
counter: 83
counter: 84
c

Zum schnelleren Verschieben werden die rotierten Bilder gezippt. Für die Modellerstellung werden sie in die erläuterte Ordnerstruktur übertragen. 

In [ ]:
!zip -r /content/drive/MyDrive/heli_train_2nd_Round.zip /content/Yolo_bbox_manipulation

  adding: content/Yolo_bbox_manipulation/ (stored 0%)
  adding: content/Yolo_bbox_manipulation/rotated_P0223_81.jpg (deflated 1%)
  adding: content/Yolo_bbox_manipulation/rotated_P1158_30.txt (deflated 62%)
  adding: content/Yolo_bbox_manipulation/rotated_P1395_201.txt (deflated 64%)
  adding: content/Yolo_bbox_manipulation/rotated_P1150_200.jpg (deflated 6%)
  adding: content/Yolo_bbox_manipulation/rotated_P1393_87.txt (deflated 67%)
  adding: content/Yolo_bbox_manipulation/rotated_P2269_206.txt (deflated 50%)
  adding: content/Yolo_bbox_manipulation/rotated_P1159_121.txt (deflated 61%)
  adding: content/Yolo_bbox_manipulation/rotated_P2226_314.txt (deflated 64%)
  adding: content/Yolo_bbox_manipulation/rotated_P1393_254.txt (deflated 67%)
  adding: content/Yolo_bbox_manipulation/rotated_P1412_121.txt (deflated 56%)
  adding: content/Yolo_bbox_manipulation/rotated_P1170_204.jpg (deflated 5%)
  adding: content/Yolo_bbox_manipulation/rotated_P0223_171.txt (deflated 61%)
  adding: conten